In [1]:
import numpy as np
import pandas as pd
from openpyxl import load_workbook
from openpyxl.pivot.fields import Missing

In [2]:
file_path = '../data/vendas-combustiveis-m3.xlsx'

In [3]:
workbook = load_workbook(file_path)
worksheet = workbook['Plan1']

In [5]:
pivot_name = 'Tabela dinâmica3'

In [6]:
# Extract the pivot table object from the worksheet
pivot_table = [p for p in worksheet._pivots if p.name == pivot_name][0]


In [7]:
# Extract a dict of all cache fields and their respective values
fields_map = {}
for field in pivot_table.cache.cacheFields:
    if field.sharedItems.count > 0:
        # take care of cases where f.v returns an AttributeError because the cell is empty
        # fields_map[field.name] = [f.v for f in field.sharedItems._fields]
        l = []
        for f in field.sharedItems._fields:
            try:
                l += [f.v]
            except AttributeError:
                l += [""]
        fields_map[field.name] = l


In [8]:
# Extract all rows from cache records. Each row is initially parsed as a dict
column_names = [field.name for field in pivot_table.cache.cacheFields]


In [9]:
rows = []
for record in pivot_table.cache.records.r:
    # If some field in the record in missing, we replace it by NaN
    record_values = [
        field.v if not isinstance(field, Missing) else np.nan for field in record._fields
    ]

    row_dict = {k: v for k, v in zip(column_names, record_values)}

    # Shared fields are mapped as an Index, so we replace the field index by its value
    for key in fields_map:
        row_dict[key] = fields_map[key][row_dict[key]]

    rows.append(row_dict)


In [10]:
df = pd.DataFrame.from_dict(rows)
df

,COMBUSTÍVEL,ANO,REGIÃO,ESTADO,Jan,Fev,Mar,Abr,Mai,Jun,Jul,Ago,Set,Out,Nov,Dez,TOTAL
0,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,RONDÔNIA,81453.67,3517.6,3681.7,4700.67,5339.2,6166.4,6539.65,7283.7,8082.85,7902.55,9383.15,9767.4,9088.8
1,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,ACRE,1483.0,11202.0,363.0,410.0,536.0,607.0,740.0,756.0,971.0,1174.0,1240.0,1439.0,1483.0
2,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,AMAZONAS,6836.3,6784.232,61443.832,3190.585,3305.0,3391.0,3637.0,4250.0,4576.0,5756.879,6228.636,6334.0,7154.2
3,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,RORAIMA,1475.3,1502.7,1531.8,13423.7,795.4,757.2,939.8,1040.6,966.0,992.9,1027.0,1083.8,1311.2
4,ÓLEO DIESEL S-10 (m3),2013.0,REGIÃO NORTE,PARÁ,40913.48,45383.5,44013.219,41975.03,441140.785,30137.8,28146.3,31280.5,33033.05,33519.88,34321.53,37168.16,41248.336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1075,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO SUL,RIO GRANDE DO SUL,0.0,0.0,10.0,10.0,0.0,10.0,,,,60.0,10.0,10.0,10.0
1076,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO CENTRO-OESTE,MATO GROSSO DO SUL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,0.0,0.0,0.0
1077,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO CENTRO-OESTE,MATO GROSSO,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,,,,90.0,45.0
1078,ÓLEO DIESEL (OUTROS ) (m3),2020.0,REGIÃO CENTRO-OESTE,GOIÁS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41.4,,,,41.4


In [68]:
df.columns

Index(['COMBUSTÍVEL', 'ANO', 'REGIÃO', 'ESTADO', 'Jan', 'Fev', 'Mar', 'Abr',
       'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez', 'TOTAL'],
      dtype='object')

In [69]:
df.shape

(4536, 17)

In [72]:
df.dtypes

COMBUSTÍVEL     object
ANO            float64
ESTADO          object
Jan             object
Fev             object
Mar             object
Abr             object
Mai             object
Jun             object
Jul             object
Ago             object
Set             object
Out             object
Nov             object
Dez             object
dtype: object

In [71]:
cols_to_drop = ['TOTAL','REGIÃO']
df.drop(columns=cols_to_drop, inplace=True)

In [74]:
dict_m = {"Jan": 1, "Fev": 2, "Mar": 3, "Abr": 4, "Mai": 5,"Jun": 6, "Jul": 7, "Ago": 8, "Set": 9, "Out": 10, "Nov": 11, "Dez": 12}

In [75]:
df = pd.melt(df, id_vars = ["COMBUSTÍVEL", "ANO", "ESTADO"], value_vars = dict_m.keys(), var_name = "month", value_name = "volume")


In [76]:
df

,COMBUSTÍVEL,ANO,ESTADO,month,volume
0,GASOLINA C (m3),2000.0,RONDÔNIA,Jan,136073.253
1,GASOLINA C (m3),2000.0,ACRE,Jan,3358.346
2,GASOLINA C (m3),2000.0,AMAZONAS,Jan,20766.918
3,GASOLINA C (m3),2000.0,RORAIMA,Jan,3716.032
4,GASOLINA C (m3),2000.0,PARÁ,Jan,29755.907
...,...,...,...,...,...
54427,GLP (m3),2020.0,RIO GRANDE DO SUL,Dez,64045.161232
54428,GLP (m3),2020.0,MATO GROSSO DO SUL,Dez,16281.139493
54429,GLP (m3),2020.0,MATO GROSSO,Dez,18321.987319
54430,GLP (m3),2020.0,GOIÁS,Dez,46850.585145


In [78]:
df['unit'] = 'm3'

from datetime import datetime
# timestamp = datetime.now()
# timestamp_series = pd.to_datetime(timestamp)
df['created_at'] = datetime.now()

In [82]:
df.rename(columns={'COMBUSTÍVEL': 'product','ESTADO': 'uf'},inplace=True)

In [83]:
df

,product,ANO,uf,month,volume,unit,created_at
0,GASOLINA C (m3),2000.0,RONDÔNIA,Jan,136073.253,m3,2023-10-10 13:32:01.584206
1,GASOLINA C (m3),2000.0,ACRE,Jan,3358.346,m3,2023-10-10 13:32:01.584206
2,GASOLINA C (m3),2000.0,AMAZONAS,Jan,20766.918,m3,2023-10-10 13:32:01.584206
3,GASOLINA C (m3),2000.0,RORAIMA,Jan,3716.032,m3,2023-10-10 13:32:01.584206
4,GASOLINA C (m3),2000.0,PARÁ,Jan,29755.907,m3,2023-10-10 13:32:01.584206
...,...,...,...,...,...,...,...
54427,GLP (m3),2020.0,RIO GRANDE DO SUL,Dez,64045.161232,m3,2023-10-10 13:32:01.584206
54428,GLP (m3),2020.0,MATO GROSSO DO SUL,Dez,16281.139493,m3,2023-10-10 13:32:01.584206
54429,GLP (m3),2020.0,MATO GROSSO,Dez,18321.987319,m3,2023-10-10 13:32:01.584206
54430,GLP (m3),2020.0,GOIÁS,Dez,46850.585145,m3,2023-10-10 13:32:01.584206


In [84]:
df.dtypes

product               object
ANO                  float64
uf                    object
month                 object
volume                object
unit                  object
created_at    datetime64[ns]
dtype: object

In [92]:
df['year_month'] = (df['ANO'].astype(int).astype(str) + '-' + df['month'].replace(dict_m).astype(str))

In [94]:
df['year_month'] = pd.to_datetime(df['year_month'], format='%Y-%m')


In [96]:
columns_order = ['year_month','uf','product','unit','volume','created_at']
df = df[columns_order]


In [99]:
df.dtypes

year_month    datetime64[ns]
uf                    object
product               object
unit                  object
volume                object
created_at    datetime64[ns]
dtype: object

In [100]:
df

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,136073.253,2023-10-10 13:32:01.584206
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346,2023-10-10 13:32:01.584206
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,20766.918,2023-10-10 13:32:01.584206
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3716.032,2023-10-10 13:32:01.584206
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,29755.907,2023-10-10 13:32:01.584206
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232,2023-10-10 13:32:01.584206
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493,2023-10-10 13:32:01.584206
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,18321.987319,2023-10-10 13:32:01.584206
54430,2020-12-01,GOIÁS,GLP (m3),m3,46850.585145,2023-10-10 13:32:01.584206


In [104]:
len(df['product'].unique().tolist())

8

In [103]:
12*21

252

In [57]:
df['ESTADO'] = df['ESTADO'].astype('string')

In [58]:
df.dtypes

COMBUSTÍVEL     string
ANO            float64
ESTADO          string
Jan             object
Fev             object
Mar             object
Abr             object
Mai             object
Jun             object
Jul             object
Ago             object
Set             object
Out             object
Nov             object
Dez             object
dtype: object

In [59]:
df['unit'] = 'm3'


In [60]:
df['unit'] = df['unit'].astype('string')

In [61]:
from datetime import datetime
timestamp = datetime.now()
timestamp_series = pd.to_datetime(timestamp)
df['created_at'] = timestamp_series

In [62]:
df.dtypes

COMBUSTÍVEL            string
ANO                   float64
ESTADO                 string
Jan                    object
Fev                    object
Mar                    object
Abr                    object
Mai                    object
Jun                    object
Jul                    object
Ago                    object
Set                    object
Out                    object
Nov                    object
Dez                    object
unit                   string
created_at     datetime64[ns]
dtype: object

In [63]:
MONTH_DICT = {
    "Jan": 1, 
    "Fev": 2, 
    "Mar": 3, 
    "Abr": 4, 
    "Mai": 5,
    "Jun": 6, 
    "Jul": 7, 
    "Ago": 8, 
    "Set": 9, 
    "Out": 10, 
    "Nov": 11, 
    "Dez": 12
}

In [64]:
df = pd.melt(df, id_vars = ["ANO", "ESTADO", "COMBUSTÍVEL", "unit", "created_at"], value_vars = MONTH_DICT.keys(), var_name = "month", value_name = "volume")


In [49]:
df2 = pd.melt(df, id_vars = ["ANO", "ESTADO", "COMBUSTÍVEL", "unit","created_at"], var_name = "month", value_name = "volume")


In [35]:
df2

,ANO,ESTADO,COMBUSTÍVEL,unit,month,volume
0,2000.0,RONDÔNIA,GASOLINA C (m3),m3,month,Jan
1,2000.0,ACRE,GASOLINA C (m3),m3,month,Jan
2,2000.0,AMAZONAS,GASOLINA C (m3),m3,month,Jan
3,2000.0,RORAIMA,GASOLINA C (m3),m3,month,Jan
4,2000.0,PARÁ,GASOLINA C (m3),m3,month,Jan
...,...,...,...,...,...,...
108859,2020.0,RIO GRANDE DO SUL,GLP (m3),m3,volume,64045.161232
108860,2020.0,MATO GROSSO DO SUL,GLP (m3),m3,volume,16281.139493
108861,2020.0,MATO GROSSO,GLP (m3),m3,volume,18321.987319
108862,2020.0,GOIÁS,GLP (m3),m3,volume,46850.585145


In [65]:
df

,ANO,ESTADO,COMBUSTÍVEL,unit,created_at,month,volume
0,2000.0,RONDÔNIA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,136073.253
1,2000.0,ACRE,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,3358.346
2,2000.0,AMAZONAS,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,20766.918
3,2000.0,RORAIMA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,3716.032
4,2000.0,PARÁ,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,Jan,29755.907
...,...,...,...,...,...,...,...
54427,2020.0,RIO GRANDE DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,64045.161232
54428,2020.0,MATO GROSSO DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,16281.139493
54429,2020.0,MATO GROSSO,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,18321.987319
54430,2020.0,GOIÁS,GLP (m3),m3,2023-10-10 13:07:50.805680,Dez,46850.585145


In [66]:
df.replace({"month": MONTH_DICT})

,ANO,ESTADO,COMBUSTÍVEL,unit,created_at,month,volume
0,2000.0,RONDÔNIA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,136073.253
1,2000.0,ACRE,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,3358.346
2,2000.0,AMAZONAS,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,20766.918
3,2000.0,RORAIMA,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,3716.032
4,2000.0,PARÁ,GASOLINA C (m3),m3,2023-10-10 13:07:50.805680,1,29755.907
...,...,...,...,...,...,...,...
54427,2020.0,RIO GRANDE DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,12,64045.161232
54428,2020.0,MATO GROSSO DO SUL,GLP (m3),m3,2023-10-10 13:07:50.805680,12,16281.139493
54429,2020.0,MATO GROSSO,GLP (m3),m3,2023-10-10 13:07:50.805680,12,18321.987319
54430,2020.0,GOIÁS,GLP (m3),m3,2023-10-10 13:07:50.805680,12,46850.585145


In [106]:
df.dtypes

year_month    datetime64[ns]
uf                    object
product               object
unit                  object
volume                object
created_at    datetime64[ns]
dtype: object

In [109]:
df['uf'] = df['uf'].astype('string')
df['product'] = df['product'].astype('string')
df['unit'] = df['unit'].astype('string')

/tmp/ipykernel_7353/1726659777.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['uf'] = df['uf'].astype('string')
/tmp/ipykernel_7353/1726659777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['product'] = df['product'].astype('string')
/tmp/ipykernel_7353/1726659777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/

In [128]:
df.dtypes

year_month    datetime64[ns]
uf                    string
product               string
unit                  string
volume               float64
created_at    datetime64[ns]
dtype: object

In [127]:
# df['volume'] = df['volume'].astype(float)
# df['volume'] = df['volume'].astype(float)
df['volume'] = pd.to_numeric(df['volume'])


/tmp/ipykernel_7353/2029536521.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['volume'] = pd.to_numeric(df['volume'])


In [116]:
import numpy as np
# df[column] = df[column].astype(np.float64)

# df['volume'] = df['volume'].astype(np.float64)

ValueError: could not convert string to float: ''

In [137]:
df

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,136073.253000,2023-10-10 13:32:01.584206
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346000,2023-10-10 13:32:01.584206
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,20766.918000,2023-10-10 13:32:01.584206
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3716.032000,2023-10-10 13:32:01.584206
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,29755.907000,2023-10-10 13:32:01.584206
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232,2023-10-10 13:32:01.584206
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493,2023-10-10 13:32:01.584206
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,18321.987319,2023-10-10 13:32:01.584206
54430,2020-12-01,GOIÁS,GLP (m3),m3,46850.585145,2023-10-10 13:32:01.584206


In [139]:
output_dir = '../data/partitioned_parquet_data'

# Specify the columns to use for partitioning
partition_cols = ['year_month']

# Save the DataFrame to Parquet with partitioning
df.to_parquet(output_dir, partition_cols=partition_cols, engine='pyarrow',compression='snappy')


In [130]:
import pyarrow as pa
import pyarrow.parquet as pq

In [131]:
tab = pa.Table.from_pandas(df)


In [140]:
pq.write_table(tab, file_path, compression='SNAPPY')


In [141]:
pq.write_table(tab, './df_pq_snappy', compression='SNAPPY')

In [143]:
df.to_parquet('./df_to_parquet_p_uf',partition_cols=['uf'],engine='pyarrow')
# df.to_parquet(output_dir, partition_cols=partition_cols, engine='pyarrow')


In [19]:
import pandas as pd

In [24]:
df_oil = pd.read_parquet('../data/oil_parquet')

In [25]:
df_oil

,year_month,uf,product,unit,volume,created_at
0,2000-01-01,RONDÔNIA,GASOLINA C (m3),m3,136073.253000,2023-10-10 19:18:41.597043
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346000,2023-10-10 19:18:41.597043
2,2000-01-01,AMAZONAS,GASOLINA C (m3),m3,20766.918000,2023-10-10 19:18:41.597043
3,2000-01-01,RORAIMA,GASOLINA C (m3),m3,3716.032000,2023-10-10 19:18:41.597043
4,2000-01-01,PARÁ,GASOLINA C (m3),m3,29755.907000,2023-10-10 19:18:41.597043
...,...,...,...,...,...,...
54427,2020-12-01,RIO GRANDE DO SUL,GLP (m3),m3,64045.161232,2023-10-10 19:18:41.597043
54428,2020-12-01,MATO GROSSO DO SUL,GLP (m3),m3,16281.139493,2023-10-10 19:18:41.597043
54429,2020-12-01,MATO GROSSO,GLP (m3),m3,18321.987319,2023-10-10 19:18:41.597043
54430,2020-12-01,GOIÁS,GLP (m3),m3,46850.585145,2023-10-10 19:18:41.597043


In [29]:
df_oil_acre = df_oil[df_oil['uf'] =='ACRE']

In [30]:
df_oil_acre

,year_month,uf,product,unit,volume,created_at
1,2000-01-01,ACRE,GASOLINA C (m3),m3,3358.346000,2023-10-10 19:18:41.597043
28,2001-01-01,ACRE,GASOLINA C (m3),m3,3345.625000,2023-10-10 19:18:41.597043
55,2002-01-01,ACRE,GASOLINA C (m3),m3,4451.197000,2023-10-10 19:18:41.597043
82,2003-01-01,ACRE,GASOLINA C (m3),m3,3963.002000,2023-10-10 19:18:41.597043
109,2004-01-01,ACRE,GASOLINA C (m3),m3,4362.524000,2023-10-10 19:18:41.597043
...,...,...,...,...,...,...
54298,2016-12-01,ACRE,GLP (m3),m3,3089.547101,2023-10-10 19:18:41.597043
54325,2017-12-01,ACRE,GLP (m3),m3,2808.168478,2023-10-10 19:18:41.597043
54352,2018-12-01,ACRE,GLP (m3),m3,3119.644928,2023-10-10 19:18:41.597043
54379,2019-12-01,ACRE,GLP (m3),m3,36850.103261,2023-10-10 19:18:41.597043


In [31]:
df_oil_acre = df_oil_acre[['year_month','volume']]

In [32]:
df_oil_acre.groupby('year_month').sum()

,volume
year_month,
2000-01-01,25317.600727
2000-02-01,61668.390364
2000-03-01,26753.502182
2000-04-01,26456.282818
2000-05-01,26580.042818
...,...
2020-08-01,17393.071130
2020-09-01,115369.506667
2020-10-01,27066.249000


In [33]:
df_oil_acre['mês'] = df_oil_acre['year_month'].dt.month
df_oil_acre['ano'] = df_oil_acre['year_month'].dt.year

/tmp/ipykernel_14679/767886557.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oil_acre['mês'] = df_oil_acre['year_month'].dt.month
/tmp/ipykernel_14679/767886557.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oil_acre['ano'] = df_oil_acre['year_month'].dt.year


In [34]:
df_oil_acre.drop(columns=['year_month','mês'],inplace=True)
# df_oil_acre.drop(columns=['mês'],inplace=True)

/tmp/ipykernel_14679/3100978454.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_oil_acre.drop(columns=['year_month','mês'],inplace=True)


In [35]:
# df_oil.drop
df_oil_acre.pivot_table(columns='ano',aggfunc='sum')

ano,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
volume,574283.034636,608945.759273,599742.127182,508244.533686,459016.074909,328305.627364,428080.449909,416900.295275,468299.539001,486315.872169,...,516504.248704,738741.174803,632884.086791,686344.655249,675278.218343,667301.859,660319.786662,523214.272217,674511.966855,465935.812


In [36]:
25317.6007272727+61668.3903636364+26753.5021818182+26456.2828181818+26580.0428181818+50320.2998181818+25658.5686363636+226545.812727273+24948.5950909091+26451.8447272727+31215.8365454545+22366.2581818182


574283.0346363637